In [16]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub

In [37]:
RESCALE_FACTOR = 1./255
BATCH_SIZE = 32
EPOCHS = 3
IMAGE_RES = 224

In [25]:
(ds_train, ds_test), ds_info = tfds.load(
                                        'cats_vs_dogs',
                                        split = ('train[:80%]', 'train[80%:]'),
                                        with_info=True,
                                        as_supervised=True)

In [26]:
def format_image(image, label):
    image = tf.image.resize(image, (IMAGE_RES, IMAGE_RES))/RESCALE_FACTOR
    return image, label


In [27]:
n_examples = ds_info.splits['train'].num_examples
n_examples

23262

In [31]:
ds_train = ds_test.shuffle(n_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)
ds_train = ds_test.map(format_image).batch(BATCH_SIZE).prefetch(1)

In [32]:
print(ds_train)
print(ds_test)

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>
<PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>


In [33]:
URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
feature_extractor = hub.KerasLayer(URL, input_shape=(IMAGE_RES, IMAGE_RES, 3))
feature_extractor.trainable = False

In [34]:
model = tf.keras.Sequential([
                             feature_extractor,
                             tf.keras.layers.Dense(2)                           
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_2 (KerasLayer)  (None, 1280)              2257984   
                                                                 
 dense_1 (Dense)             (None, 2)                 2562      
                                                                 
Total params: 2,260,546
Trainable params: 2,562
Non-trainable params: 2,257,984
_________________________________________________________________


In [35]:
model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

In [38]:
history = model.fit(ds_train,
                    epochs=EPOCHS,
                    validation_data=ds_test)

Epoch 1/3
  2/146 [..............................] - ETA: 2:22 - loss: 0.5654 - accuracy: 0.7344

Corrupt JPEG data: 162 extraneous bytes before marker 0xd9


 14/146 [=>............................] - ETA: 2:10 - loss: 0.5849 - accuracy: 0.7009

Corrupt JPEG data: 252 extraneous bytes before marker 0xd9


 58/146 [==========>...................] - ETA: 1:36 - loss: 0.5746 - accuracy: 0.7047

Corrupt JPEG data: 214 extraneous bytes before marker 0xd9


 91/146 [=================>............] - ETA: 1:01 - loss: 0.5771 - accuracy: 0.7047

Corrupt JPEG data: 1403 extraneous bytes before marker 0xd9


146/146 [==============================] - ETA: 0s - loss: 0.5691 - accuracy: 0.7135

ValueError: in user code:

    File "/Users/daviyokogawa/.pyenv/versions/3.8.6/lib/python3.8/site-packages/keras/engine/training.py", line 1525, in test_function  *
        return step_function(self, iterator)
    File "/Users/daviyokogawa/.pyenv/versions/3.8.6/lib/python3.8/site-packages/keras/engine/training.py", line 1514, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/daviyokogawa/.pyenv/versions/3.8.6/lib/python3.8/site-packages/keras/engine/training.py", line 1507, in run_step  **
        outputs = model.test_step(data)
    File "/Users/daviyokogawa/.pyenv/versions/3.8.6/lib/python3.8/site-packages/keras/engine/training.py", line 1471, in test_step
        y_pred = self(x, training=False)
    File "/Users/daviyokogawa/.pyenv/versions/3.8.6/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/daviyokogawa/.pyenv/ver